In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

groq_api_key= os.getenv('GROQ_API_KEY')


In [97]:
llm = ChatGroq(model='Llama3-8b-8192', api_key=groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021C41D09DF0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021C41BBCA10>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
os.environ['HF_TOKEN']=os.getenv('HF_TOKEN')

In [6]:
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\kaleb\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import bs4

In [35]:
loader = WebBaseLoader(
    web_path="https://lilianweng.github.io/posts/2023-06-23-agent/",
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    )
)

In [36]:
docs=loader.load()

In [38]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [43]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False
)

In [41]:
text= text_splitter.split_documents(docs)

In [44]:
Vector_store = Chroma.from_documents(text, embedding=embeddings)

In [45]:
retriver=Vector_store.as_retriever()
retriver

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000021C41672720>, search_kwargs={})

In [98]:
### Prompt template

system_prompt=(
    """You are a helpful assistant to answer questions
    Use the following pieces of retrieved context to answer
    if you don't have the answer in the cotext say i dont know the answer \n\n
    """
    "{context}"
)


prompt=ChatPromptTemplate.from_messages(
    [("system", system_prompt),
    ("human", "{input}"),]
)

In [99]:
document_stuff_chain = create_stuff_documents_chain(llm, prompt)
retrival_chain = create_retrieval_chain(retriver, document_stuff_chain)

In [100]:
respon=retrival_chain.invoke({"input":"What is self reflection"})

In [101]:
respon['answer']

'According to the provided context, self-reflection is "created by showing two-shot examples to LLM and each example is a pair of". Additionally, it is mentioned that self-reflection is a "vital aspect that allows autonomous agents to improve iteratively by refining" their actions.'

## Adding Chat history

In [102]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

In [136]:
contextualized_q_system_prompt=(
    """Use the chat history as a context and the users input to answer the question
    if the context doesn't include any information that you can use to answer the question
    answer i don't know"""
)

context_q_prompt = ChatPromptTemplate(
    [
        ("system", contextualized_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)

In [137]:
history_aware_retriever = create_history_aware_retriever(llm, retriver, context_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000021C41672720>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessag

In [130]:
context_prompt = ChatPromptTemplate(
    [
        ("system", contextualized_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)

In [138]:
prompt=ChatPromptTemplate.from_messages(
    [("system", system_prompt),
     MessagesPlaceholder("chat_history"),
    ("human", "{input}"),]
)

In [139]:
qna_chain= create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, qna_chain)

In [140]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

In [141]:
chat_history=[]
question="What is self-reflection"
respo=rag_chain.invoke({"input":question, "chat_history":chat_history})

In [142]:
respo['answer']

'According to the retrieved context, self-reflection is "a vital aspect that allows autonomous agents to improve iteratively by refining". It is also mentioned that self-reflection is created by showing two-shot examples to LLM (Large Language Model) and each example is a pair of. Additionally, it\'s mentioned that self-reflection involves "reflection and refinement: The agent can do self-criticism and self-reflection over past actions".'

In [145]:
chat_history=[]
question="What is self reflection"
respo1 = rag_chain.invoke({'input':question, "chat_history":chat_history})
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=respo1['answer'])
]
)
question2="How do you accomplish it?"
respo2 = rag_chain.invoke({'input':question2, 'chat_history':chat_history})

In [146]:
respo2['answer']

'According to the context, self-reflection can be accomplished through mechanisms like self-reflection, which allows an agent to learn from past mistakes.'

In [158]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [159]:
store={}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store['session_id'] = ChatMessageHistory()
    return store['session_id']


In [160]:
conversational_rag_chain =  RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

In [162]:
config_rag1={"configurable":{"session_id":"abc1423"}}

In [163]:
resp = conversational_rag_chain.invoke(
    {"input":"What is Task decomposition?"},
    config=config_rag1,
)

In [164]:
resp['answer']

'Based on the provided context, Task decomposition is the process of breaking down large tasks into smaller, manageable subgoals.'

In [165]:
resp2 = conversational_rag_chain.invoke(
    {"input":"What are the common ways accomplishing it?"},
    config=config_rag1,
)

In [166]:
resp2['answer']

'According to the context, one common way to accomplish task decomposition is by using a simple prompting technique, such as "Steps for XYZ.\\n1." or "What are the steps for XYZ?".'

In [167]:
resp3 = conversational_rag_chain.invoke(
    {"input":"What what was the first question i asked?"},
    config=config_rag1,
)

In [168]:
resp3['answer']


'Your first question was "What is Task decomposition?"'